In [119]:
import cv2
import numpy as np

In [120]:
img = cv2.imread('./imgs/frame_933.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [121]:
img.shape

(80, 60, 3)

In [122]:
# dump as c header file, H x W x C
H, W, C = img.shape
with open('./frame_933.jpg.h', 'w') as f:
  f.write('#pragma once\n')
  f.write('#define H {}\n'.format(H))
  f.write('#define W {}\n'.format(W))
  f.write('#define C {}\n'.format(C))
  f.write('unsigned char img[H][W][C] = {\n')
  
  for i in range(H):
    f.write('  {')
    for j in range(W):
      f.write('{')
      for k in range(C):
        f.write('{}, '.format(img[i, j, k]))
      f.write('}, ')
    f.write('},\n')
  
  f.write('};')
  

In [125]:
import numpy as np
import queue   

def bfs(img, start):
  calculate_distance = lambda x, y: np.sqrt(np.sum((x - y) ** 2))
  dirs = [[1, 0], [-1, 0], [0, 1], [0, -1]]
  dst = np.zeros(img.shape[:-1])
  q = queue.Queue()
  q.put(start)
  
  while not q.empty():
    current = q.get()
    for dir in dirs:
      next_pos = (current[0] + dir[0], current[1] + dir[1])
      if 0 <= next_pos[0] < img.shape[0] and 0 <= next_pos[1] < img.shape[1]:
        if dst[next_pos] == 0 and calculate_distance(img[next_pos], img[current]) < 12:
          dst[next_pos] = 1
          q.put(next_pos)

  return dst


def get_edge(img):
  height, width = 80, 60
  edgesL = []
  for i in range(height):
    for j in range(width - 1):
      if img[i, j] == 0 and img[i, j+1] == 1:
        edgesL.append([j, i])
        break
      
  return np.array(edgesL)

In [127]:
img = cv2.imread('./imgs/frame_933.jpg')

dst = bfs(img, (img.shape[0]//2-1, img.shape[1]//2-1))
edgesL = get_edge(dst)

x, y = edgesL[:,0], edgesL[:,1]
A = np.vstack([y, np.ones(len(y))]).T

AT = A.T
ATMA = AT @ A

ATMAI = np.linalg.inv(ATMA)

ATMAIAT = ATMAI @ AT

ATMAIATx = ATMAIAT @ x

print(ATMAIATx)

[-0.07946336 18.27382181]


In [130]:
AT

array([[32., 33., 34., 35., 36., 37., 38., 39., 40., 41., 42., 43., 44.,
        45., 46., 47., 48., 49.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.]])

In [129]:
ATMAIAT.shape

(2, 18)